# Forecast con Moirai-MoE-Base (Salesforce/moirai-moe-1.0-R-base) + Yahoo Finance (descarga por ventanas)

**Notebook generado automáticamente.**

Este cuaderno descarga datos de Yahoo Finance por *ventanas* de tiempo (robusto a índices con zonas horarias y posibles columnas MultiIndex), normaliza columnas OHLCV, y realiza *forecasting probabilístico* con **Moirai-MoE-Base** usando **Uni2TS + GluonTS**. Genera un CSV y una imagen con el pronóstico.

### Requisitos sugeridos (entorno estable)

```bash
pip install "numpy==1.26.4" "pandas==2.1.4"
pip install "torch==2.3.1" --index-url https://download.pytorch.org/whl/cpu
pip install "uni2ts==1.2.0" "gluonts==0.14.3" yfinance matplotlib huggingface_hub tqdm
```

> Puedes ajustar a GPU instalando el build de PyTorch con CUDA que corresponda a tu sistema. En Windows, si usas `pip`, sigue la guía de PyTorch.


In [1]:
# Imports básicos y configuración general
import warnings, re, time
from pathlib import Path
from typing import Optional, Dict, List

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from tqdm import tqdm

from gluonts.dataset.common import ListDataset
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule

warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 120


c:\Users\Administrator\Desktop\PROYECTOS\FORECASTING\venv_moirai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Utilidades de fechas y frecuencia


In [2]:
def _allowed_days_and_step(interval: str):
    interval = interval.lower()
    if interval in ("1m", "2m"): return 30, 5
    if interval in ("5m", "15m", "30m"): return 60, 10
    if interval in ("60m", "1h", "90m"): return 730, 60
    if interval in ("1d", "5d", "1wk"): return 10000, 365 * 2
    return 3650, 180

def _to_pd_freq(interval: str) -> str:
    m = interval.lower()
    if m.endswith("m"): return f"{m[:-1]}min"
    if m.endswith("h"): return f"{m[:-1]}H"
    if m.endswith("d"): return "D"
    if m.endswith("wk"): return "W"
    return "D"

def _to_utc(ts) -> pd.Timestamp:
    ts = pd.Timestamp(ts)
    if ts.tzinfo is None: return ts.tz_localize("UTC")
    return ts.tz_convert("UTC")


## Normalización robusta de columnas (maneja MultiIndex, `adj_close` → `close`, tokens como `AAPL_Close`, etc.)


In [3]:
_OHLCV_KEYS = ("open", "high", "low", "close", "adj_close", "volume")

def _flatten_columns(df: pd.DataFrame) -> List[str]:
    if isinstance(df.columns, pd.MultiIndex):
        cols = [
            "_".join([str(x) for x in tup if x is not None and f"{x}".strip() != ""]).strip()
            for tup in df.columns.values
        ]
    else:
        cols = [str(c) for c in df.columns]
    return cols

def _normalize_ohlcv_columns(df: pd.DataFrame, debug: bool=False, tag: str="") -> pd.DataFrame:
    if df is None or df.empty:
        return df

    # 1) Aplanar y asignar
    flat_cols = _flatten_columns(df)
    df = df.copy()
    df.columns = flat_cols

    # 2) Normalizar nombres básicos y asignar (minúsculas, espacios->'_')
    norm_cols = [c.strip().lower().replace(" ", "_") for c in df.columns]
    df.columns = norm_cols

    # 3) Tokenizar y agrupar candidatos por clave
    buckets: Dict[str, List[str]] = {k: [] for k in _OHLCV_KEYS}
    for col in df.columns:
        tokens = re.split(r"[^a-z0-9]+", col)
        ts = [t for t in tokens if t]
        s = set(ts)
        if "open" in s: buckets["open"].append(col)
        if "high" in s: buckets["high"].append(col)
        if "low"  in s: buckets["low"].append(col)
        if "close" in s and "adj" not in s: buckets["close"].append(col)
        if "close" in s and "adj" in s: buckets["adj_close"].append(col)
        if "volume" in s: buckets["volume"].append(col)

    # Intento de coincidencia exacta si algún bucket está vacío
    for col in df.columns:
        if col in _OHLCV_KEYS and col not in sum(buckets.values(), []):
            buckets[col].append(col)

    # 4) Elegir el primer candidato por clave
    picks: Dict[str, str] = {}
    for k in _OHLCV_KEYS:
        if buckets[k]:
            picks[k] = buckets[k][0]

    # 5) Si no hay 'close' pero hay 'adj_close', usarla como close
    if "close" not in picks and "adj_close" in picks:
        picks["close"] = picks["adj_close"]

    if debug:
        print(f"[normalize{(':'+tag) if tag else ''}] flat_cols={flat_cols}")
        print(f"[normalize{(':'+tag) if tag else ''}] norm_cols={norm_cols}")
        print(f"[normalize{(':'+tag) if tag else ''}] picks={picks}")

    keep_order = [k for k in ["open","high","low","close","volume"] if k in picks]
    if not keep_order:
        return df

    out = pd.DataFrame(index=df.index.copy())
    for k in keep_order:
        out[k] = pd.to_numeric(df[picks[k]], errors="coerce")
    return out


## Descarga ventana-a-ventana (Yahoo Finance)


In [4]:
def download_yf_windowed(
    ticker: str, interval: str, start=None, end=None,
    want_max=True, tz="UTC", max_retries=3, sleep=1.0, debug: bool=False
) -> pd.DataFrame:
    allowed_days, step_days = _allowed_days_and_step(interval)

    # --- Fechas robustas ---
    if end is None: end = pd.Timestamp.now(tz="UTC")
    else: end = _to_utc(end)

    if start is None:
        if want_max: start = end - pd.Timedelta(days=allowed_days * 50)
        else: start = end - pd.Timedelta(days=allowed_days)
    else:
        start = _to_utc(start)

    dfs = []
    cur = start
    pbar = tqdm(total=None, desc=f"Descargando {ticker} {interval} por ventanas")

    while cur < end:
        win_end = min(cur + pd.Timedelta(days=step_days), end)
        retries = 0
        got = None
        while retries <= max_retries:
            try:
                df = yf.download(
                    ticker, interval=interval,
                    start=cur, end=win_end,
                    progress=False, auto_adjust=False, prepost=False, threads=True,
                    group_by="column"  # intenta evitar MultiIndex por ticker
                )
                if isinstance(df, pd.DataFrame) and not df.empty:
                    got = df.copy()
                break
            except Exception:
                retries += 1
                time.sleep(sleep * (2 ** retries))
        if got is not None and not got.empty:
            got = _normalize_ohlcv_columns(got, debug=debug, tag=f"{cur.date()}->{win_end.date()}")
            if debug:
                print(f"[window] {cur} -> {win_end} shape={got.shape} cols={list(got.columns)}")
            dfs.append(got)
        cur = win_end
        pbar.update(1)

    pbar.close()

    if len(dfs) == 0:
        raise RuntimeError("No se descargaron datos (posible combinación inválida ticker/intervalo).")

    data = pd.concat(dfs).sort_index()
    data = data.loc[~data.index.duplicated(keep="last")]

    # normalización final por seguridad
    data = _normalize_ohlcv_columns(data, debug=debug, tag="final")

    # Índice tz-aware -> UTC -> tz destino
    if data.index.tz is None: data.index = data.index.tz_localize("UTC")
    else: data.index = data.index.tz_convert("UTC")
    data = data.tz_convert(tz)

    # astype seguro y dropna all
    for c in data.columns:
        data[c] = pd.to_numeric(data[c], errors="coerce")
    data = data.dropna(how="all")
    return data


## Helpers de zona horaria y *plotting*


In [5]:
def _to_naive_utc(idx: pd.DatetimeIndex) -> pd.DatetimeIndex:
    if idx.tz is None: idx = idx.tz_localize("UTC")
    else: idx = idx.tz_convert("UTC")
    return idx.tz_localize(None)

def make_plot(history: pd.Series, pred_index: pd.DatetimeIndex, mean: np.ndarray,
              p10: Optional[np.ndarray] = None, p90: Optional[np.ndarray] = None,
              title: str = "Forecast con Moirai-MoE", out_png: Optional[Path] = None) -> Path:
    plt.figure(figsize=(11, 5))
    plt.plot(history.index, history.values, label="Histórico (contexto)")
    plt.plot(pred_index, mean, label="Pronóstico (media)")
    if p10 is not None and p90 is not None:
        plt.fill_between(pred_index, p10, p90, alpha=0.2, label="PI 80%")
    plt.title(title); plt.xlabel("Tiempo"); plt.ylabel("Valor"); plt.legend(); plt.tight_layout()
    out_png = out_png or Path("forecast_plot.png"); plt.savefig(out_png, dpi=150); plt.close()
    return out_png


## Función principal de *forecast*


In [6]:
def run_forecast(
    ticker: str,
    interval: str,
    start: Optional[str],
    end: Optional[str],
    pred_len: int,
    ctx_len: int,
    patch_size: int,
    num_samples: int,
    batch_size: int,
    column: str,
    out_dir: Path,
    tz_out: str,
    debug: bool=False
):
    out_dir.mkdir(parents=True, exist_ok=True)

    # 1) Descarga
    raw = download_yf_windowed(
        ticker=ticker, interval=interval, start=start, end=end,
        want_max=True, tz=tz_out, max_retries=3, sleep=1.0, debug=debug
    )
    if raw is None or raw.empty:
        raise RuntimeError("No se obtuvieron datos desde Yahoo. Revisa ticker/intervalo/fechas.")

    if debug:
        print(f"[raw] shape={raw.shape} cols={list(raw.columns)} idx=[{raw.index[0]} -> {raw.index[-1]}]")

    # 2) Columna objetivo
    col_key = column.strip().lower().replace(" ", "_")
    if col_key not in raw.columns:
        raise ValueError(f"La columna '{column}' no existe. Disponibles: {list(raw.columns)}")
    s = raw[col_key].copy()  # Serie con tz

    # 3) Frecuencia uniforme
    freq = _to_pd_freq(interval)
    full_idx = pd.date_range(start=s.index[0], end=s.index[-1], freq=freq, tz=s.index.tz)
    s = s.reindex(full_idx).ffill().bfill()

    # 4) Contexto
    if ctx_len > 0 and len(s) > ctx_len: s_ctx = s.iloc[-ctx_len:].copy()
    else: s_ctx = s.copy()

    if len(s_ctx) < max(8, pred_len + 1):
        raise RuntimeError(f"Pocos datos ({len(s_ctx)}) para {ticker} en {interval}. Ajusta --start/--ctx_len.")

    # 5) Dataset GluonTS (naive UTC + 1D float32)
    start_naive_utc = _to_naive_utc(s_ctx.index)[0]
    target_1d = s_ctx.to_numpy(dtype="float32")
    input_ds = ListDataset([{"start": start_naive_utc, "target": target_1d}], freq=freq)

    # 6) Modelo Moirai-MoE
    module = MoiraiMoEModule.from_pretrained("Salesforce/moirai-moe-1.0-R-base")
    model = MoiraiMoEForecast(
        module=module,
        prediction_length=pred_len,
        context_length=len(s_ctx),
        patch_size=patch_size,
        num_samples=num_samples,
        target_dim=1,
        feat_dynamic_real_dim=0,
        past_feat_dynamic_real_dim=0,
    )
    predictor = model.create_predictor(batch_size=batch_size)

    # 7) Predicción
    forecasts = predictor.predict(input_ds)
    forecast = next(iter(forecasts))

    # 8) Índice futuro y cuantiles
    start_pred = pd.Timestamp(getattr(forecast.start_date, "to_timestamp", lambda: forecast.start_date)())
    pred_index_naive = pd.date_range(start=start_pred, periods=pred_len, freq=freq)
    pred_index = pred_index_naive.tz_localize("UTC").tz_convert(s_ctx.index.tz)

    mean = forecast.mean
    try:
        p10 = forecast.quantile(0.1); p90 = forecast.quantile(0.9)
    except Exception:
        p10 = forecast.quantile("0.1"); p90 = forecast.quantile("0.9")

    # 9) Guardar
    out_csv = out_dir / "forecast.csv"
    pd.DataFrame({"timestamp": pred_index, "mean": mean, "p10": p10, "p90": p90}) \
        .set_index("timestamp").to_csv(out_csv, float_format="%.6f")

    title = f"{ticker} {interval} | Moirai-MoE-Base (pred_len={pred_len}, ctx_len={len(s_ctx)})"
    out_png = make_plot(s_ctx, pred_index, mean, p10, p90, title=title, out_png=out_dir / "forecast_plot.png")

    # 10) Resumen
    print("[OK] Pronóstico generado")
    print(f"  - Ticker:        {ticker}")
    print(f"  - Intervalo:     {interval}  (freq={freq})")
    print(f"  - TZ salida:     {tz_out}")
    print(f"  - Contexto:      {len(s_ctx)} muestras")
    print(f"  - Predicción:    {pred_len} pasos")
    print(f"  - Patch size:    {patch_size}")
    print(f"  - Muestras:      {num_samples}")
    print(f"  - CSV:           {out_csv.resolve()}")
    print(f"  - Gráfico PNG:   {out_png.resolve()}")


## Parámetros (edita y ejecuta)


In [19]:
TICKER   = "BTC-USD"          # Ej.: "AAPL", "BTC-USD", "^GSPC"
INTERVAL = "1d"            # 1m 2m 5m 15m 30m 60m 90m 1h 1d 5d 1wk 1mo 3mo
START    = "2015-01-01"    # o None
END      = None            # o "YYYY-MM-DD"
PRED_LEN = 30
CTX_LEN  = 10000
PATCH    = 16
SAMPLES  = 100
BATCH    = 32
COLUMN   = "close"         # close/open/high/low/volume
OUT_DIR  = Path("outputs_moirai_moe")
TZ_OUT   = "America/Mexico_City"
DEBUG    = False           # True para imprimir detalles de normalización/ventanas


## Ejecutar pronóstico


In [20]:
run_forecast(
    ticker=TICKER,
    interval=INTERVAL,
    start=START,
    end=END,
    pred_len=PRED_LEN,
    ctx_len=CTX_LEN,
    patch_size=PATCH,
    num_samples=SAMPLES,
    batch_size=BATCH,
    column=COLUMN,
    out_dir=OUT_DIR,
    tz_out=TZ_OUT,
    debug=DEBUG,
)


Descargando BTC-USD 1d por ventanas: 6it [00:00, 25.75it/s]


[OK] Pronóstico generado
  - Ticker:        BTC-USD
  - Intervalo:     1d  (freq=D)
  - TZ salida:     America/Mexico_City
  - Contexto:      3905 muestras
  - Predicción:    30 pasos
  - Patch size:    16
  - Muestras:      100
  - CSV:           C:\Users\Administrator\Desktop\PROYECTOS\FORECASTING\FORECASTING\outputs_moirai_moe\forecast.csv
  - Gráfico PNG:   C:\Users\Administrator\Desktop\PROYECTOS\FORECASTING\FORECASTING\outputs_moirai_moe\forecast_plot.png
